In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import shap


url = "https://github.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/raw/main/Demand_Forecsting/random_forest.csv"
data = pd.read_csv(url)


print(data.describe().round(2))


X = data.drop(columns=["Week", "Demand"])
y = data["Demand"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R²: {r2:.2f}")


feature_importances = rf_model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({"Feature": features, "Importance": feature_importances}).sort_values(by="Importance", ascending=False)


plt.figure(figsize=(8, 5))
plt.barh(importance_df["Feature"], importance_df["Importance"], color="skyblue")
plt.title("Feature Importance")
plt.xlabel("Importance")
plt.ylabel("Features")
plt.gca().invert_yaxis()
plt.show()


residuals = y_test - y_pred
plt.figure(figsize=(8, 5))
plt.scatter(y_pred, residuals, alpha=0.7)
plt.axhline(0, color="red", linestyle="--")
plt.title("Residual Analysis")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.show()


correlation_matrix = data.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()


rf_limited_depth = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf_limited_depth.fit(X_train, y_train)
y_pred_limited_depth = rf_limited_depth.predict(X_test)
print(f"R² with limited depth: {r2_score(y_test, y_pred_limited_depth):.2f}")


explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)


shap.summary_plot(shap_values, X_test, plot_type="bar")


data["Price_Weather_Interaction"] = data["Price_Index"] * data["Weather_Spikes"]
X_interaction = data.drop(columns=["Week", "Demand"])
X_train_interaction, X_test_interaction, y_train, y_test = train_test_split(X_interaction, y, test_size=0.2, random_state=42)

rf_interaction = RandomForestRegressor(n_estimators=100, random_state=42)
rf_interaction.fit(X_train_interaction, y_train)
y_pred_interaction = rf_interaction.predict(X_test_interaction)
print(f"R² with interaction term: {r2_score(y_test, y_pred_interaction):.2f}")


data["Lagged_Demand_1"] = data["Demand"].shift(1)
data["Lagged_Demand_2"] = data["Demand"].shift(2)
data = data.dropna()
X_lagged = data.drop(columns=["Week", "Demand"])
y_lagged = data["Demand"]
X_train_lagged, X_test_lagged, y_train_lagged, y_test_lagged = train_test_split(X_lagged, y_lagged, test_size=0.2, random_state=42)

rf_lagged = RandomForestRegressor(n_estimators=100, random_state=42)
rf_lagged.fit(X_train_lagged, y_train_lagged)
y_pred_lagged = rf_lagged.predict(X_test_lagged)
print(f"R² with lagged variables: {r2_score(y_test_lagged, y_pred_lagged):.2f}")


X_new_conditions = X_test.copy()
X_new_conditions["Weather_Spikes"] += 10
X_new_conditions["Economic_Indicator"] -= 20
y_new_conditions = rf_model.predict(X_new_conditions)
print(f"Predicted demand under new conditions:\n{y_new_conditions[:5]}")


data_monthly = data.groupby(data.index // 4).mean()
X_monthly = data_monthly.drop(columns=["Demand"])
y_monthly = data_monthly["Demand"]
X_train_monthly, X_test_monthly, y_train_monthly, y_test_monthly = train_test_split(X_monthly, y_monthly, test_size=0.2, random_state=42)

rf_monthly = RandomForestRegressor(n_estimators=100, random_state=42)
rf_monthly.fit(X_train_monthly, y_train_monthly)
y_pred_monthly = rf_monthly.predict(X_test_monthly)
print(f"R² with monthly data: {r2_score(y_test_monthly, y_pred_monthly):.2f}")


